In [1]:
# Contract Risk Analyzer - Exploration Notebook
# Purpose:
# 1. Understand legal dataset
# 2. Prepare text for ML
# 3. Train and evaluate a baseline risk classifier

In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

print("All libraries imported successfully")

All libraries imported successfully
